<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, EdX<br>
            Dr. Eric Van Dusen <br>
        Akhil Venkatesh <br>
</table>

# Lecture Notebook 1.2a: Creating a Demand Curve

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from datascience import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

For our iteration of this class in August of 2022, we asked students in the room to make a bid representing how much they would be willing to pay for different goods: Masks, Burritos, iPhones, and GreekTix. We attempted to then derive a demand curve based on these results.

### Let's load in a few demand curves based on this survey taken from class! Here's how the survey looked:
https://docs.google.com/forms/d/e/1FAIpQLScVfl0IiCoFQwrKsHrICKAJsA1mZn7GZCjQC_97YRnqXJCP2A/viewform


### You can find the survey responses here:
https://docs.google.com/spreadsheets/d/1Izl5h_PXIph-Dk4PrsZCBWq4KQFiIoA_xh2odVo3b9A/edit#gid=1309761632

### For this example we downloaded these results as a csv file that we will read in here

In [ ]:
demand_table = Table.read_table("Data88EDemandSurvey-Fall22.csv")
demand_table = demand_table.drop('Timestamp')
demand_table

Let's try graphing all our different responses!

In [ ]:
demand_table.hist("Masks", bins = 8, left_end = min(demand_table.column("Masks")), 
                  right_end = max(demand_table.column("Masks")))


In [ ]:
demand_table.hist("Burrito", bins = 8, left_end = min(demand_table.column("Burrito")), 
                  right_end = max(demand_table.column("Burrito")))

In [ ]:
demand_table.hist("GreekTix", bins = 8, left_end = min(demand_table.column("GreekTix")), 
                  right_end = max(demand_table.column("GreekTix")))


In [ ]:
demand_table.hist("iPhone", bins = 8, left_end = min(demand_table.column("iPhone")), 
                  right_end = max(demand_table.column("iPhone")))


## OK Looks good, but it doesnt look like a Demand Curve yet?

**Let's focus on the burritos first.** How many people are willing to buying a gourment burrito at any given price?   
We can assume that a person would be willing to buy the good at a price less than their bid price.  

In [ ]:
BurritosTable = demand_table.select('Burrito')
BurritosTable

In [ ]:
# Count how many people are in each answer pool
BurritosTable.group("Burrito")

In [ ]:
# Create a bar plot
table = BurritosTable.group("Burrito")

def plot_histogram(data, bins, title="Title", x_label = "Price", y_label = "Count"):
    plt.bar(bins, data, edgecolor="brown", align="center", width = 2)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()
    return 

burrito_bins = table.column(0) # Select column using method call
burrito_data = table['count'] # Select column using indexing
burrito_title = "Demand of Burritos according to different prices"

plot_histogram(burrito_data, burrito_bins, burrito_title)

In the visualization above, the height of each bar isn't quite right - someone who is willing to pay \\$10 for a burrito will also pay \\$2.5 for the same burrito. 

Since people will pay up to their maximum bid price, let's change the visualization and the table to instead consider the **total demand** at any given price. 
We can do this in the following three steps: 
1. Flip the order of the count column
2. Apply a cumulative function of all prices below
3. Flip it back

In [ ]:
Q0 = BurritosTable.group("Burrito").column("count")
Q0

This array Q0 is the number of people willing to pay at each price from lowest to highest

### Step 1: Flip the order of the count column


In [ ]:
Q1 = np.flip(Q0)
Q1

This array Q1 is the number of people willing to pay at each price from highest to lowest

### Step 2: Sum the number of people willing to pay that price or higher


In [ ]:
# Step 2: Apply a cumulative function of all prices below
Q2 = np.cumsum(Q1)
Q2

This array Q2 is the total number of people willing to pay at each price point from highest to lowest

### Step 3: Flip it back again so that we can match it to the prices 


In [ ]:
# Step 3: Flip it back
Q3 = np.flip(Q2)
Q3

In [ ]:
# Or we can do this in just one line
Qdemand = np.flip(np.cumsum(np.flip(BurritosTable.group("Burrito").column("count"))))
Qdemand

In [ ]:
# Combine the price column and demand column into a single table
DemandBurr= Table().with_columns([
    'priceBurr', [2.5, 5, 7.5, 10.00, 12.5, 15,17.5], # those are the prices
    'Qdemand', Qdemand
])
DemandBurr

In [ ]:
# Create a bar plot
burrito_Qdemand_bins = DemandBurr.column('priceBurr') # Select column using method call
burrito_Qdemand_data = DemandBurr['Qdemand'] # Select column using indexing
burrito_Qdemand_title = "Quantity demanded of Burritos with different prices"
Qdemand_x_label = "Price of Burrito"
Qdemand_y_label = "Quantity demanded"

plot_histogram(burrito_Qdemand_data, burrito_Qdemand_bins, burrito_Qdemand_title, Qdemand_x_label, Qdemand_y_label)

Let's continue by  looking at the demand for greek theater tickets. How many people would buy greek theater tickets at a given price? Let's assume that a person would be willing to buy the good at a price less than their bid price.  

In [ ]:
# This is a column of bid values for greek theater tickets that have been inputted. 
greektix = demand_table.select('GreekTix')
greektix

In [ ]:
#Let's create a table summarizing the quantity of bid values! First we'll need to define our two columns!
prices = np.array([2.50, 5.00, 7.50, 10.00, 12.50, 15.00, 17.50, 20.00])
quantities = np.flip(np.cumsum(np.flip(greektix.group("GreekTix").column(1))))
quantities

In case you're wondering how we came up with the quantities values: We perform np.flip first so that we can cumulatively sum up the quantities. We then perform np.cumsum to calculate the cumulative sums. Because np.cumsum reverts the order of the array to ascending order, we perform np.flip again to get the array in descending order (that is, from 99 to 4).


In [ ]:
#Now we can use pd.DataFrame to actually create the columns with their respective labels and values!
greektix_demand = pd.DataFrame({'Price': prices, 'Quantity': quantities})
greektix_demand

In [ ]:
# Let's graph our results
plt.scatter(greektix_demand["Quantity"], greektix_demand["Price"])
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for two tickets for an awesome concert at the Greek Theater');

Now let's find the slope and intercept of the line of best fit. The cell below defines some functions that you'll learn about in the later portions of Data 8.

In [ ]:
std_units = lambda a: (a - np.mean(a)) / np.std(a)
corr = lambda x, y: np.mean(std_units(x) * std_units(y))
slope = lambda x, y: corr(x, y) * np.std(y) / np.std(x)
intercept = lambda x, y: np.mean(y) - slope(x, y) * np.mean(x)

In [ ]:
slope(greektix_demand["Quantity"], greektix_demand["Price"])

In [ ]:
intercept(greektix_demand["Quantity"], greektix_demand["Price"])

Feel free to explore this in your own time: Let's create demand curves for our other products as well!

In [ ]:
#Gourmet Burrito
prices_burrito = pd.DataFrame({'price':[2.50, 5, 7.50, 10, 12.5, 15,17.5,20]})

burritos = demand_table.select('Burrito')
burritosByPrice = burritos.group("Burrito")
bbp = burritosByPrice.to_df()
gb = (
    prices_burrito
    .merge(bbp, left_on='price', how='left', right_on='Burrito')
    .fillna(0).drop('Burrito', axis=1)
)

burritos_table = Table.from_df(gb)
Q_demand_burrito = np.flip(np.cumsum(np.flip(burritos_table.group("price", sum).column(1))))

gb_demand = Table().with_columns(
    'price', prices_burrito.price, 
    'quantity', Q_demand_burrito
)

burrito_slope = slope(gb_demand["quantity"], gb_demand["price"])
burrito_intercept = intercept(gb_demand["quantity"], gb_demand["price"])
print("Slope: " + str(burrito_slope))
print("Intercept: " +  str(burrito_intercept))

In [ ]:
#Masks
prices_masks = pd.DataFrame({'price':[0.25, 0.50, .75, 1.00, 1.25, 1.50, 1.75, 2.00]})

masks = demand_table.select('Masks')
masksByPrice = masks.group("Masks")
tbp = masksByPrice.to_df()
ms = (
    prices_masks
    .merge(tbp, left_on='price', how='left', right_on='Masks')
    .fillna(0).drop('Masks', axis=1)
)

masks_table = Table.from_df(ms)
Q_demand_masks = np.flip(np.cumsum(np.flip(masks_table.group("price", sum).column(1))))

ms_demand = Table().with_columns(
    'price', prices_masks.price, 
    'quantity', Q_demand_masks
)

masks_slope = slope(ms_demand["quantity"], ms_demand["price"])
masks_intercept = intercept(ms_demand["quantity"], ms_demand["price"])
print("Slope: " + str(masks_slope))
print("Intercept: " +  str(masks_intercept))

In [ ]:
#Iphone 14
prices_iphone = pd.DataFrame({'price':[250, 500, 750, 1000, 1250, 1500,1750,2000, 2250, 2500, 2750, 3000]})

iphones = demand_table.select('iPhone')
iphonesByPrice = iphones.group("iPhone")
ibp = iphonesByPrice.to_df()
iphone14 = (
    prices_iphone
    .merge(ibp, left_on='price', how='left', right_on="iPhone")
    .fillna(0).drop("iPhone", axis=1)
)

iphones_table = Table.from_df(iphone14)
Q_demand_iphones = np.flip(np.cumsum(np.flip(iphones_table.group("price", sum).column(1))))

iphone14_demand = Table().with_columns(
    'price', prices_iphone.price, 
    'quantity', Q_demand_iphones
)

iphones_slope = slope(iphone14_demand["quantity"], iphone14_demand["price"])
iphones_intercept = intercept(iphone14_demand["quantity"], iphone14_demand["price"])
print("Slope: " + str(iphones_slope))
print("Intercept: " +  str(iphones_intercept))

Comparing the demand curves for our four products, what similarities or differences do you notice? In particular, think about what the slopes of the curves might reveal to us about consumer preferences. 

In [ ]:
ms_demand.scatter("quantity", "price")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for a pack of surgical masks');

gb_demand.scatter("quantity", "price")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Gourmet Burritos');

#greektix_demand.scatter("quantity", "price")
#plt.xlabel('Quantity')
#plt.ylabel('Price')
#plt.title('Demand for Greek Theatre Tickets');

iphone14_demand.scatter("quantity", "price")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for iPhone14');